In [ ]:
import sys
sys.path.append("..")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from utils import DataGenerator, read_annotation_lines
from models import Yolov4
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import xml.etree.ElementTree as ET
from glob import glob
import cv2
import numpy as np

tf.config.experimental.list_physical_devices()

import matplotlib.pyplot as plt
%matplotlib inline

import random
import time

%load_ext autoreload
%autoreload 2

In [ ]:
BASE_DIR = 'datasets/some_pascalvoc_dataset'

PATH_IMG = os.path.join(BASE_DIR, 'JPEGImages/')
PATH_CLASS = os.path.join(BASE_DIR, 'labels.txt')
PATH_XML = os.path.join(BASE_DIR, 'Annotations/')

PATH_ANN = os.path.join(BASE_DIR, 'annotations.txt')

## Convert xml into txt

In [ ]:
'''loads the classes'''
def get_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [ ]:
classes = get_classes(PATH_CLASS)
assert len(classes) > 0, 'no class names detected!'
print(f'num classes: {len(classes)}')

In [ ]:
# output file
list_file = open(PATH_ANN, 'w')

for path in glob(os.path.join(PATH_XML, '*.xml')):
    in_file = open(path)

    # Parse .xml file
    tree = ET.parse(in_file)
    root = tree.getroot()
    # Write object information to .txt file
    file_name = root.find('filename').text
    print(file_name)
    list_file.write(file_name)
    for obj in root.iter('object'):
        cls = obj.find('name').text 
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))
    list_file.write('\n')
list_file.close()

In [ ]:
train_lines, val_lines = read_annotation_lines(PATH_ANN, test_size = 0.2)

## Custom data generator

In [ ]:
from imgaug import augmenters as iaa
import imgaug as ia

seq = iaa.Sequential([
    iaa.OneOf([
        iaa.Sometimes(0.1, iaa.HorizontalFlip(1)),
        iaa.Sometimes(0.1, iaa.VerticalFlip(1)),
        iaa.Sometimes(0.1, iaa.Rot90([1])),
        iaa.Sometimes(0.1, iaa.Rot90([2])),
    ]),

    iaa.Sometimes(0.2, iaa.OneOf([
        iaa.Crop(px=(5, 16)),
        iaa.Affine(
            scale={'x': (1, 1.2), 'y': (1, 1.2)},
            # translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-5, 5),
            shear=(-10, 10)
        ),

    ])),

    iaa.Sometimes(0.1, ia.augmenters.color.Grayscale(alpha=1)),
    iaa.Sometimes(0.1, iaa.AddToHue((-20, 20))),
    #     iaa.Sometimes(0.1, ia.augmenters.color.AddToHueAndSaturation((-20, 20))),
    iaa.Sometimes(0.1, iaa.AdditiveGaussianNoise(scale=0.01 * 255)),
    iaa.Sometimes(0.1, iaa.GammaContrast((0.6, 1.7))),
], random_order=False)

seq = iaa.Sequential([
    iaa.Rot90([1]),
], random_order=False)

In [ ]:
train_generator_1 = DataGenerator(train_lines, PATH_CLASS, PATH_IMG, batch_size=1, shuffle=True, augmentors=seq)
for (image, *_, boxes), _ in train_generator_1:
    
    image = np.squeeze(image)
    boxes = np.squeeze(boxes)
    
    for box in boxes:
        if np.all(box == 0):
            break
        x1, y1, w, h = box
        x1 = int(x1 - w // 2)
        x2 = int(x1 + w)
        y1 = int(y1 - h // 2)
        y2 = int(y1 + h)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(image)
    
    break

## Get data

In [ ]:
train_lines, val_lines = read_annotation_lines(PATH_ANN, test_size = 0.2)
#print(train_lines)

data_gen_train = DataGenerator(train_lines, PATH_CLASS, PATH_IMG)
data_gen_val = DataGenerator(val_lines, PATH_CLASS, PATH_IMG)
#print(data_gen_train)

In [ ]:
weight_path = '../yolov4.weights'
model = Yolov4(weight_path=weight_path, class_name_path=PATH_CLASS)

## Training

In [ ]:
epochs = 100

In [ ]:
path = f'your_save_path/'

if not os.path.exists(path):
    os.makedirs(path)
 
# filepath = path + 'epoch_{epoch:02d}-val_loss-{val_loss:.4f}.h5'
filepath = path + 'model.h5'

callbacks = [
    ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, save_best_only = True,
                    save_weights_only = False, mode = 'auto', period = 1),
    ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 5, verbose = 1),
    EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 10),
    TensorBoard(log_dir = path + '/tensorboard', histogram_freq = 0, write_graph = False, write_images = False),
]

In [ ]:
results = model.fit(data_gen_train, 
          initial_epoch=0,
          epochs=epochs, 
          val_data_gen=data_gen_val,
          callbacks=callbacks)       

In [ ]:
model.save_model(path + f'yolov4_e{epochs}.h5')

In [ ]:
model.inference_model.save(path + f'yolov4_e{epochs}_inf.h5')

## Predict

use get_detection_data_np function in models.py/predict_img

In [ ]:
model_path = 'yolov4_100.h5'

In [ ]:
model = Yolov4(class_name_path=PATH_CLASS)
model.load_model(model_path)

In [ ]:
filenames = glob(PATH_IMG + '/*.*')
len(filenames)

In [ ]:
ix = random.randint(0, len(filenames) - 1)
# ix = 0
print(ix, filenames[ix])
t1 = time.time()
img = cv2.imread(filenames[ix], cv2.IMREAD_UNCHANGED)[:, :, ::-1]
img = np.array(img)
boxes, scores, labels = model.predict(filenames[ix], random_color=False, plot_img=True, show_text=False)
t2 = time.time()

print(f'Prediction time : {t2 - t1}')

scale = max(img.shape[0:2]) / 416
line_width = int(2 * scale)

h, w = img.shape[:2]

for box, score, label in zip(boxes, scores, labels):
    
    print(label, score, box)
    x1, y1, x2, y2 = box
    x1 = int(x1 * w)
    x2 = int(x2 * w)
    y1 = int(y1 * h)
    y2 = int(y2 * h)
    color = (255, 0, 0)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, line_width)
    text = f'{label} {score:.2f}'
    font = cv2.FONT_HERSHEY_DUPLEX
    font_scale = max(0.3 * scale, 0.3)
    thickness = max(int(1 * scale), 1)
    (text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=thickness)[0]
    cv2.rectangle(img, (x1 - line_width//2, y1 - text_height), (x1 + text_width, y1), color, cv2.FILLED)
    cv2.putText(img, text, (x1, y1), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)
    
fig = plt.figure(figsize=(16, 16))
plt.imshow(img)

In [ ]:
type(detections)

## Evaluate

In [ ]:
path = f'your_save_path/'

In [ ]:
gt_folder_path = os.path.join(path, 'map', 'gt_folder')
if not os.path.exists(gt_folder_path):
    os.makedirs(gt_folder_path)
pred_folder_path = os.path.join(path, 'map', 'pred_folder')
if not os.path.exists(pred_folder_path):
    os.makedirs(pred_folder_path)
    
model.export_gt(PATH_ANN, gt_folder_path)

In [ ]:
model.export_prediction(PATH_ANN, pred_folder_path, PATH_IMG, bs=1)

In [ ]:
temp_json_folder_path = os.path.join(path, 'map', 'json')
if not os.path.exists(temp_json_folder_path):
    os.makedirs(temp_json_folder_path)
output_files_path = os.path.join(path, 'map')
model.eval_map(gt_folder_path, pred_folder_path, temp_json_folder_path, output_files_path)

## Freeze graph

In [ ]:
import os
import sys

import argparse

from pathlib import Path

import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

from keras import backend as K
from keras.models import load_model

In [ ]:
def setKerasOptions():
    K._LEARNING_PHASE = tf.constant(0)
    K.set_learning_phase(False)
    K.set_learning_phase(0)
    K.set_image_data_format('channels_last')


def getInputParameters():
    parser = argparse.ArgumentParser()
    parser.add_argument('--input_model', '-m', required=True, type=str, help='Path to Keras model.')
    parser.add_argument('--num_outputs', '-no', required=False, type=int, help='Number of outputs. 1 by default.', default=1)

    return parser


def export_keras_to_tf(input_model, output_model, num_output, custom_metric=None):
    print('Loading Keras model: ', input_model)
    
    if custom_metric is None:
        keras_model = load_model(input_model)
    else:
        keras_model = load_model(input_model, custom_objects=custom_metric)

    print(keras_model.summary())

    predictions = [None] * num_output
    predrediction_node_names = [None] * num_output

    for i in range(num_output):
        predrediction_node_names[i] = 'output_node' + str(i)
        predictions[i] = tf.identity(keras_model.outputs[i], name=predrediction_node_names[i])

    sess = K.get_session()

    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), predrediction_node_names)
    infer_graph = graph_util.remove_training_nodes(constant_graph) 

    graph_io.write_graph(infer_graph, '.', output_model, as_text=False)
    
def export_keras_to_tf_model(keras_model, output_model, num_output):
    print('Loading Keras model: ', input_model)

    # print(keras_model.summary())

    predictions = [None] * num_output
    predrediction_node_names = [None] * num_output

    for i in range(num_output):
        predrediction_node_names[i] = 'output_node' + str(i)
        predictions[i] = tf.identity(keras_model.outputs[i], name=predrediction_node_names[i])

    sess = K.get_session()

    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), predrediction_node_names)
    infer_graph = graph_util.remove_training_nodes(constant_graph) 

    graph_io.write_graph(infer_graph, '.', output_model, as_text=False)

In [ ]:
base_dir = 'your_save_path/'

model_name = 'yolov4_e10.h5'

input_model = os.path.join(base_dir, model_name)
num_output = 1

output_model = os.path.join(base_dir, str(Path(input_model).name) + '.pb')
# custom_metric = {'iou': iou}
custom_metric = None

output_model

In [ ]:
predrediction_node_names = export_keras_to_tf_model(model.inference_model, output_model, num_output)

print('Ouput nodes are:', predrediction_node_names)
print('Saved as TF frozen model to: ', output_model)